In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

In [43]:
data = pd.read_csv('dados/results-survey292284(4).csv')

In [3]:
# Parse the XML file
tree = ET.parse('dados/survey_292284_pt-BR.xml')

# Get the root element
root = tree.getroot()

In [32]:
sections = [element for element in root.iter() if element.tag == 'section']
section_names = [section.get('id') for section in sections]
section_names

['1230', '698', '699', '697', '695', '694', '701', '700', '693']

In [36]:
root.find('.//section[@id="700"]').findall('.//subQuestion')

[<Element 'subQuestion' at 0x000002258863CBD0>,
 <Element 'subQuestion' at 0x000002258863CCC0>,
 <Element 'subQuestion' at 0x000002258863CD60>,
 <Element 'subQuestion' at 0x000002258863CE00>,
 <Element 'subQuestion' at 0x000002258863CEA0>,
 <Element 'subQuestion' at 0x000002258863CF40>,
 <Element 'subQuestion' at 0x0000022588640040>,
 <Element 'subQuestion' at 0x00000225886400E0>,
 <Element 'subQuestion' at 0x0000022588640180>,
 <Element 'subQuestion' at 0x0000022588640220>,
 <Element 'subQuestion' at 0x00000225886407C0>,
 <Element 'subQuestion' at 0x00000225886408B0>,
 <Element 'subQuestion' at 0x00000225886409A0>,
 <Element 'subQuestion' at 0x0000022588640A40>,
 <Element 'subQuestion' at 0x0000022588640B30>]

In [37]:
varmap = {}
for section in [root.find(f'.//section[@id="{section_id}"]') for section_id in ['700', '693', '694']]:
    for subquestion in section.findall('.//subQuestion'):
        var_name = subquestion.get('varName')
        text = subquestion.find('text').text if subquestion.find('text') is not None else ""
        varmap[var_name] = text

    

In [38]:
sub_questions_dict = {
    'adc1_SQ001': 'palestra público geral',
    'adc1_SQ002': 'curso público externo',
    'adc1_SQ003': 'aula oficina escola básica',
    'adc1_SQ004': 'evento comissão técnica conselho profissional',
    'adc1_SQ005': 'atividade Pint of Science',
    'adc1_SQ006': 'atividade Dia Ciência Semana Nacional Ciência Tecnologia',
    'adc1_SQ007': 'evento associação ONG movimento social',
    'adc1_SQ008': 'artigo revista público geral',
    'adc1_SQ009': 'entrevista jornal revista público geral',
    'adc1_SQ010': 'livro capítulo divulgação científica',
    'adc1_SQ011': 'release imprensa programa TV rádio',
    'adc1_SQ012': 'audiência pública poder legislativo',
    'adc1_SQ013': 'visita guiada museu',
    'adc1_SQ014': 'programa mídias digitais',
    'adc1_SQ015': 'conteúdo canal próprio mídias digitais',
    'adc3_SQ001': 'assessoria comunicação geral universidade instituto pesquisa',
    'adc3_SQ002': 'assessoria comunicação laboratório unidade acadêmica',
    'adc3_SQ003': 'mídia jornalistas',
    'adc3_SQ004': 'profissionais museus',
    'adc6_SQ001': 'diretamente',
    'adc6_SQ002': 'via assessoria comunicação geral universidade instituto pesquisa',
    'adc6_SQ003': 'via assessoria comunicação laboratório unidade acadêmica'
}


In [39]:
for key in varmap.keys():
    if key in sub_questions_dict:
        varmap[key] = sub_questions_dict[key]

In [40]:
colunas = [key for key in varmap.keys() if key.startswith('adc1')]

In [41]:
colunas = [coluna.replace("_", "[") + "]" for coluna in colunas]

In [45]:
data.columns = data.columns.to_series().replace(varmap).str.replace("\[", "_").str[:-1]

C:\Users\MAPERE~1\AppData\Local\Temp/ipykernel_16720/3419636055.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.to_series().replace(varmap).str.replace("\[", "_").str[:-1]


In [47]:
for coluna in data.columns:
    if coluna in sub_questions_dict:
        novo_nome = sub_questions_dict[coluna]
        data.rename(columns={coluna: novo_nome}, inplace=True)

In [48]:
varmap

{'VAL01_SQ001': 'Igreja ou grupo religioso',
 'VAL01_SQ002': 'Associação esportiva ou recreativa',
 'VAL01_SQ003': 'Associação artística, musical ou educacional',
 'VAL01_SQ004': 'Sindicato',
 'VAL01_SQ005': 'Partido político',
 'VAL01_SQ006': 'Associação de defesa do meio ambiente',
 'VAL01_SQ007': 'Associação profissional, ou seja, um grupo de profissionais voluntários da mesma área com interesses em comum',
 'VAL01_SQ008': 'Associação humanitária ou de caridade',
 'VAL01_SQ009': 'Associação de consumidores',
 'VAL01_SQ010': 'Grupo de autoajuda, grupo de ajuda mútua',
 'VAL02_SQ001': 'Assinou um abaixo assinado',
 'VAL02_SQ002': 'Participou de algum boicote',
 'VAL02_SQ003': 'Participou de manifestações pacíficas',
 'VAL02_SQ004': 'Participou de greves',
 'VAL02_SQ005': 'Participou de qualquer outro ato de protesto',
 'adc1_SQ001': 'palestra público geral',
 'adc1_SQ002': 'curso público externo',
 'adc1_SQ003': 'aula oficina escola básica',
 'adc1_SQ004': 'evento comissão técnica con

In [29]:
data.to_csv('data_tratado.csv', index=False)